In [2]:
import os
import pandas as pd
import urllib
import datetime
import time



In [3]:
time.strftime("%m%d%Y")

'05222016'

In [4]:
start = time.time()
print "hello Jalal Jahir what"
print "hello Jalal Jahir is a GREAT Guy"
end = time.time()
processtime = end - start
print "{0: .3}".format(processtime)
print float(processtime)

hello Jalal Jahir what
hello Jalal Jahir is a GREAT Guy
 0.0
0.0


In [5]:
print os.getcwd()

C:\Users\Jalal\Documents\python_proj\Projects\2016\wa_esd_emp


In [6]:
current_dir = os.getcwd()

In [7]:
esd_sa_url = "https://fortress.wa.gov/esd/employmentdata/docs/economic-reports/wa-historical-employment-seasonally-adjusted.xlsx"
sa_filename = esd_sa_url[-49:-5]+"_"+time.strftime("%m%d%Y")+esd_sa_url[-5:]
def download_esd_sa():
    start = time.time()
    esdfile = urllib.URLopener()
    esdfile.retrieve(esd_sa_url, os.getcwd()+"/"+ sa_filename)
    end = time.time()
    processtime = end - start
    print "{0: .3}".format(processtime)
    

In [8]:
download_esd_sa()

 2.17


In [13]:
help (urllib)

Help on module urllib:

NAME
    urllib - Open an arbitrary URL.

FILE
    c:\users\jalal\anaconda2\lib\urllib.py

DESCRIPTION
    See the following document for more info on URLs:
    "Names and Addresses, URIs, URLs, URNs, URCs", at
    http://www.w3.org/pub/WWW/Addressing/Overview.html
    
    See also the HTTP spec (from which the error codes are derived):
    "HTTP - Hypertext Transfer Protocol", at
    http://www.w3.org/pub/WWW/Protocols/
    
    Related standards and specs:
    - RFC1808: the "relative URL" spec. (authoritative status)
    - RFC1738 - the "URL standard". (authoritative status)
    - RFC1630 - the "URI spec". (informational status)
    
    The object returned by URLopener().open(file) will differ per
    protocol.  All you know is that is has methods read(), readline(),
    readlines(), fileno(), close() and info().  The read*(), fileno()
    and close() methods work like those of open files.
    The info() method returns a mimetools.Message object which can b

In [14]:
# Check the number of worksheet in the .xlsx file & list the name of the worksheets
#included in the analysis

def check_worksheet():
    esd_sa_file = pd.ExcelFile(os.getcwd()+"/"+ sa_filename)
    esd_sa_ws_master = {sheet_name: esd_sa_file.parse(sheet_name) 
              for sheet_name in esd_sa_file.sheet_names}

    a_list = []
    count = 0
    count_total = 0
    for i in esd_sa_ws_master:
        if i == "Seattle MSA":
            a_list.append(i)
            count = count+1
            count_total = count_total+1
        elif i == "Bremerton MSA":
            a_list.append(i)
            count = count+1
            count_total = count_total+1
        elif i == "Tacoma MSA":
            a_list.append(i)
            count = count+1
            count_total = count_total+1
        else:
            count_total = count_total+1


    print "List of worksheets to be included in the analysis: "

    count_ws = 0
    for x in sorted(a_list):
        count_ws = count_ws+1 
        print str(count_ws)+". "+ x

    print "Total "+str(count)+" "+"out of "+ str(count_total)+ " Worrksheets are in the PSRC region."

check_worksheet()

List of worksheets to be included in the analysis: 
1. Bremerton MSA
2. Seattle MSA
3. Tacoma MSA
Total 3 out of 12 Worrksheets are in the PSRC region.


In [94]:
## EXAMPLE: Read the worksheet into dataframe from a excel file:

#esd_sa_file = pd.ExcelFile(os.getcwd()+"/"+ sa_filename)
#df_sa_kin_sno = esd_sa_file.parse("Seattle MSA", header =1).head()

In [64]:
#read the required worksheets into dataframes

df_sa_kin_sno = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Seattle MSA", header = 1, index_col=[1])
df_sa_kit = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Bremerton MSA", header = 1, index_col=[1])
df_sa_pie = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Tacoma MSA", header = 1, index_col=[1])


In [16]:
type(df_sa_pie)

pandas.core.frame.DataFrame

In [17]:
df_sa_kin_sno.head()

,NAICS CELL,1990-01-01 00:00:00,1990-02-01 00:00:00,1990-03-01 00:00:00,1990-04-01 00:00:00,1990-05-01 00:00:00,1990-06-01 00:00:00,1990-07-01 00:00:00,1990-08-01 00:00:00,1990-09-01 00:00:00,...,2015-07-01 00:00:00,2015-08-01 00:00:00,2015-09-01 00:00:00,2015-10-01 00:00:00,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00
NAICS INDUSTRY,,,,,,,,,,,,,,,,,,,,,
Total Nonfarm,0,1096300,1098400,1101600,1103100,1106100,1111200,1113900,1119000,1115500,...,1596400,1602200,1606100,1606300,1608600,1614000,1619100,1627000,1629200,1634100
Total Private,5000000,951400,952200,956300,957600,960200,964400,966700,967500,967100,...,1383400,1386100,1390600,1391800,1393700,1399100,1406100,1412000,1412900,1417600
Goods Producing,6000000,277800,276800,278100,277300,277400,277900,278800,278100,277500,...,258900,257400,257500,257100,257700,259500,258700,259500,260100,259500
Mining and Logging,10000000,1900,1900,1900,2000,2000,2000,2000,2000,2000,...,800,700,800,800,800,800,800,800,800,800
Construction,20000000,61900,61800,62600,63200,63700,63800,63200,62900,62600,...,86300,85500,85700,86600,87100,88600,89000,90900,91700,91100


In [65]:
#transpose the dataframe

df_sa_kin_sno = df_sa_kin_sno.transpose()

In [66]:
#transposing caused the date column to be the index column, so
#extracting the date information to another column named "date_stamp"

df_sa_kin_sno['date_stamp'] = df_sa_kin_sno.index

In [67]:
#deleting the first data row (not the header) containing NAICS info 
#entirely for formatting purposes 

df_sa_kin_sno = df_sa_kin_sno.ix[1:]

In [68]:
#Resetting the index of the dataframe
#as we already copied the date info into another column, nothing to worry

df_sa_kin_sno = df_sa_kin_sno.reset_index(drop =True)

#renaming the index colum from "NAICS Industy" to "id" 

df_sa_kin_sno.columns.names = ["id"]

In [69]:
#get a column for year and one for month (note the foramting for month)
df_sa_kin_sno["year"] = df_sa_kin_sno["date_stamp"].dt.year
df_sa_kin_sno["month"] = df_sa_kin_sno["date_stamp"].dt.strftime("%b")

#get the month column to string (not essential)
df_sa_kin_sno["month"] = df_sa_kin_sno["month"].astype("str")

In [70]:
df_sa_kin_sno.tail()

id,Total Nonfarm,Total Private,Goods Producing,Mining and Logging,Construction,Construction of Buildings,Heavy and Civil Engineering Construction,Specialty Trade Contractors,Manufacturing,Durable Goods Mfg,...,Membership Associations and Organizations,Government,Federal Government,Total State Government,State Government Educational Services,Total Local Government,Local Government Educational Services,date_stamp,year,month
311,1614000,1399100,259500,800,88600,22900,8100,57600,170100,142300,...,27300,214900,22100,66100,48900,126700,55400,2015-12-01 00:00:00,2015,Dec
312,1619100,1406100,258700,800,89000,23000,8000,58000,168900,141100,...,27800,213000,22100,64400,47400,126500,55200,2016-01-01 00:00:00,2016,Jan
313,1627000,1412000,259500,800,90900,23400,8200,59300,167800,139900,...,28000,215000,22100,65800,48700,127100,55400,2016-02-01 00:00:00,2016,Feb
314,1629200,1412900,260100,800,91700,23400,8200,60100,167600,139300,...,27800,216300,22000,65800,48600,128500,55700,2016-03-01 00:00:00,2016,Mar
315,1634100,1417600,259500,800,91100,24400,8300,58400,167600,139300,...,28100,216500,22000,65500,48300,129000,55900,2016-04-01 00:00:00,2016,Apr


In [71]:
#get the list of column names, any of the options below works just fine
# list(df_sa_kin_sno)
# list(df_sa_kin_sno.columns.values)
kin_sno_list = df_sa_kin_sno.columns.values.tolist()


In [72]:
kin_sno_list[17]

u'        Service Providing'

In [73]:
kin_sno_list[62]

u'    Government'

In [74]:
kin_sno_list[65]

u'                    State Government Educational Services'

In [75]:
kin_sno_list[69]

'year'

In [76]:
df_sa_kin_sno["King & Snohomish: Total NonFarm"] = df_sa_kin_sno["Total Nonfarm"] 
df_sa_kin_sno["King & Snohomish: Consruction"] = df_sa_kin_sno[kin_sno_list[3]] + df_sa_kin_sno[kin_sno_list[4]]
df_sa_kin_sno["King & Snohomish: FIRE"] = df_sa_kin_sno[kin_sno_list[33]]
df_sa_kin_sno["King & Snohomish: Manufacturing"] = df_sa_kin_sno[kin_sno_list[8]]
df_sa_kin_sno["King & Snohomish: Retail"] = df_sa_kin_sno[kin_sno_list[20]]
df_sa_kin_sno["King & Snohomish: Service"] = df_sa_kin_sno[kin_sno_list[17]] - df_sa_kin_sno[kin_sno_list[18]] - df_sa_kin_sno[kin_sno_list[33]] - df_sa_kin_sno[kin_sno_list[62]]                                                     
df_sa_kin_sno["King & Snohomish: WTU"] = df_sa_kin_sno[kin_sno_list[18]] - df_sa_kin_sno[kin_sno_list[20]]
df_sa_kin_sno["King & Snohomish: Government"] = df_sa_kin_sno[kin_sno_list[62]] - df_sa_kin_sno[kin_sno_list[65]] - df_sa_kin_sno[kin_sno_list[67]]
df_sa_kin_sno["King & Snohomish: Education"] = df_sa_kin_sno[kin_sno_list[65]] + df_sa_kin_sno[kin_sno_list[67]]

In [77]:
#df_sa_kin_sno["Total Sector Jobs"] = df_sa_kin_sno.ix[:,72:].sum(axis=1)

In [78]:
#df_sa_kin_sno["Total Private Sum"] = df_sa_kin_sno.ix[:,72:-3].sum(axis= 1)

In [79]:
df_sa_kin_sno = df_sa_kin_sno.drop(df_sa_kin_sno.columns[:69], axis = 1)
#df_sa_kin_sno = df_sa_kin_sno.drop(df_sa_kin_sno.columns[range(69)], axis = 1)

In [80]:
df_sa_kin_sno = df_sa_kin_sno.drop(df_sa_kin_sno.index[[315]])

In [81]:
df_sa_kin_sno.tail()

id,year,month,King & Snohomish: Total NonFarm,King & Snohomish: Consruction,King & Snohomish: FIRE,King & Snohomish: Manufacturing,King & Snohomish: Retail,King & Snohomish: Service,King & Snohomish: WTU,King & Snohomish: Government,King & Snohomish: Education
310,2015,Nov,1608600,87900,83100,169800,169400,757400,126100,110400,104500
311,2015,Dec,1614000,89400,82900,170100,169700,759800,127200,110600,104300
312,2016,Jan,1619100,89800,84300,168900,173500,761900,127700,110400,102600
313,2016,Feb,1627000,91700,84000,167800,174300,766700,127500,110900,104100
314,2016,Mar,1629200,92500,84300,167600,173800,765400,129300,112000,104300


In [82]:
if df_sa_kin_sno["King & Snohomish: Total NonFarm"].all() == df_sa_kin_sno.ix[:,3:].sum(axis=1).all():
    print "King & Snohomish Counties: All industry sectors adds up to total NonFarm jobs"
else:
    print "ERROR"

King & Snohomish County: All industry sectors adds up to total NonFarm jobs


In [83]:
df_sa_kit = pd.read_excel(os.getcwd()+"/"+ sa_filename, sheetname="Bremerton MSA", header = 1, index_col=[1])
df_sa_kit = df_sa_kit.transpose()
df_sa_kit['date_stamp'] = df_sa_kit.index
df_sa_kit = df_sa_kit.ix[1:]
df_sa_kit = df_sa_kit.reset_index(drop =True)
df_sa_kit.columns.names = ["id"]
df_sa_kit["year"] = df_sa_kit["date_stamp"].dt.year
df_sa_kit["month"] = df_sa_kit["date_stamp"].dt.strftime("%b")

kit_list = df_sa_kit.columns.values.tolist()

df_sa_kit["Kitsap: Total NonFarm"] = df_sa_kit["Total Nonfarm"] 
df_sa_kit["Kitsap: Consruction"] = df_sa_kit[kit_list[3]]
#add new column for FIRE with zero as values with index aligned with the dataframe
df_sa_kit["Kitsap: FIRE"] = pd.Series([0 for x in range(len(df_sa_kit.index))], index=df_sa_kit.index)
df_sa_kit["Kitsap: Manufacturing"] = df_sa_kit[kit_list[4]]
df_sa_kit["Kitsap: Retail"] = df_sa_kit[kit_list[8]]
df_sa_kit["Kitsap: Service"] = df_sa_kit[kit_list[5]] - df_sa_kit[kit_list[7]] - df_sa_kit[kit_list[11]]
df_sa_kit["Kitsap: WTU"] = df_sa_kit[kit_list[7]] - df_sa_kit[kit_list[8]]
df_sa_kit["Kitsap: Government"] = df_sa_kit[kit_list[11]]
df_sa_kit["Kitsap: Education"] = pd.Series([0 for x in range(len(df_sa_kit.index))], index=df_sa_kit.index)

if df_sa_kit["Kitsap: Total NonFarm"].all() == df_sa_kit.ix[:,3:].sum(axis=1).all():
    print "Kitsap County: All industry sectors adds up to total NonFarm jobs"
else:
    print "ERROR"

df_sa_kit = df_sa_kit.drop(df_sa_kit.columns[:16], axis = 1)    
    
    
df_sa_kit.tail()

Kitsap County: All industry sectors adds up to total NonFarm jobs


id,year,month,Kitsap: Total NonFarm,Kitsap: Consruction,Kitsap: FIRE,Kitsap: Manufacturing,Kitsap: Retail,Kitsap: Service,Kitsap: WTU,Kitsap: Government,Kitsap: Education
310,2015,Nov,88200,4600,0,2400,11000,37400,1600,31200,0
311,2015,Dec,88400,4400,0,2600,11100,35000,3600,31700,0
312,2016,Jan,88300,4200,0,2700,10900,37000,1600,31900,0
313,2016,Feb,88300,4300,0,2700,11000,36600,1700,32000,0
314,2016,Mar,88400,4400,0,2600,10800,35300,3100,32200,0


In [58]:
kit_list[15]

'date_stamp'

In [54]:
df_sa_kit["Kitsap: Total NonFarm"] = df_sa_kit["Total Nonfarm"] 
df_sa_kit["Kitsap: Consruction"] = df_sa_kit[kit_list[3]]
df_sa_kit["Kitsap: FIRE"] = pd.Series([0 for x in range(len(df_sa_kit.index))], index=df_sa_kit.index)
df_sa_kit["Kitsap: Manufacturing"] = df_sa_kit[kit_list[4]]
df_sa_kit["Kitsap: Retail"] = df_sa_kit[kit_list[8]]
df_sa_kit["Kitsap: Service"] = df_sa_kit[kit_list[5]] - df_sa_kit[kit_list[7]] - df_sa_kit[kit_list[11]]
df_sa_kit["Kitsap: WTU"] = df_sa_kit[kit_list[7]] - df_sa_kit[kit_list[8]]
df_sa_kit["Kitsap: Government"] = df_sa_kit[kit_list[11]]
df_sa_kit["Kitsap: Education"] = pd.Series([0 for x in range(len(df_sa_kit.index))], index=df_sa_kit.index)

In [55]:
df_sa_kit.tail()

id,Total Nonfarm,Total Private,Goods Producing,"Mining, Logging, and Construction",Manufacturing,Service Providing,Private Service Providing,"Trade, Transportation, and Utilities",Retail Trade,Professional and Business Services,...,month,Kitsap: Total NonFarm,Kitsap: Consruction,Kitsap: FIRE,Kitsap: Manufacturing,Kitsap: Retail,Kitsap: Service,Kitsap: WTU,Kitsap: Government,Kitsap: Education
310,88200,57000,7000,4600,2400,81200,50000,12600,11000,7000,...,Nov,88200,4600,0,2400,11000,37400,1600,31200,0
311,88400,56700,7000,4400,2600,81400,49700,14700,11100,6900,...,Dec,88400,4400,0,2600,11100,35000,3600,31700,0
312,88300,56400,6900,4200,2700,81400,49500,12500,10900,7100,...,Jan,88300,4200,0,2700,10900,37000,1600,31900,0
313,88300,56300,7000,4300,2700,81300,49300,12700,11000,7100,...,Feb,88300,4300,0,2700,11000,36600,1700,32000,0
314,88400,56200,7000,4400,2600,81400,49200,13900,10800,7100,...,Mar,88400,4400,0,2600,10800,35300,3100,32200,0


In [56]:
if df_sa_kit["Kitsap: Total NonFarm"].all() == df_sa_kit.ix[:,3:].sum(axis=1).all():
    print "Hudasdrray"
else:
    print "ERROR"

Hudasdrray


In [538]:
a =["a", 'b','C',"d","222"]

In [539]:
b = [1,2,5,6,9,8,16,21]

In [541]:
a[2:]

['C', 'd', '222']

In [382]:
b[2]+b[4]-b[6]

-2

In [657]:
time.strftime("%b")

'May'

In [484]:
a =5
b = 4
c= 2

In [485]:
if a == b+c:
    pass
else:
    print "ERROR"

ERROR


In [247]:
# df_sa_king.drop(df_sa_king.columns[[1]], axis = 1).head()

In [76]:
xl_file = pd.ExcelFile(os.getcwd()+"/"+ sa_filename)

In [77]:
dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [81]:
dfs["Yakima MSA"].head()

,Yakima MSA Quarterly Benchmarked,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 307,Unnamed: 308,Unnamed: 309,Unnamed: 310,Unnamed: 311,Unnamed: 312,Unnamed: 313,Unnamed: 314,Unnamed: 315,Unnamed: 316
0,NAICS CELL,NAICS INDUSTRY,1990-01-01 00:00:00,1990-02-01 00:00:00,1990-03-01 00:00:00,1990-04-01 00:00:00,1990-05-01 00:00:00,1990-06-01 00:00:00,1990-07-01 00:00:00,1990-08-01 00:00:00,...,2015-06-01 00:00:00,2015-07-01 00:00:00,2015-08-01 00:00:00,2015-09-01 00:00:00,2015-10-01 00:00:00,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00
1,00000000,Total Nonfarm,63600,63600,64100,64300,64500,65000,65000,65000,...,82700,82400,84300,83900,84400,84200,84900,84900,85000,85300
2,05000000,Total Private,51700,51700,52100,52200,52300,52600,52400,52600,...,65600,64800,65600,65600,66200,66100,66500,66900,67200,67600
3,06000000,Goods Producing,12200,12100,12500,12400,12600,12800,12800,12800,...,12400,12200,12700,12900,12900,12500,12200,12200,12100,12300
4,15000000,"Mining, Logging, and Construction",2400,2300,2400,2400,2500,2500,2500,2600,...,3800,3800,3800,3800,3800,3800,3700,3400,3400,3600


In [62]:
dfs["Seattle MSA"].head()

,Seattle MSA Quarterly Benchmarked,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 307,Unnamed: 308,Unnamed: 309,Unnamed: 310,Unnamed: 311,Unnamed: 312,Unnamed: 313,Unnamed: 314,Unnamed: 315,Unnamed: 316
0,NAICS CELL,NAICS INDUSTRY,1990-01-01 00:00:00,1990-02-01 00:00:00,1990-03-01 00:00:00,1990-04-01 00:00:00,1990-05-01 00:00:00,1990-06-01 00:00:00,1990-07-01 00:00:00,1990-08-01 00:00:00,...,2015-06-01 00:00:00,2015-07-01 00:00:00,2015-08-01 00:00:00,2015-09-01 00:00:00,2015-10-01 00:00:00,2015-11-01 00:00:00,2015-12-01 00:00:00,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00
1,00000000,Total Nonfarm,1096300,1098400,1101600,1103100,1106100,1111200,1113900,1119000,...,1591500,1590500,1594000,1594100,1594600,1598900,1602600,1607700,1615600,1619900
2,05000000,Total Private,951400,952200,956300,957600,960200,964400,966700,967500,...,1376800,1382600,1392400,1385300,1380200,1382000,1390300,1394000,1400200,1402600
3,06000000,Goods Producing,277800,276800,278100,277300,277400,277900,278800,278100,...,258600,261300,260900,260600,254500,252900,254100,251900,252700,254200
4,10000000,Mining and Logging,1900,1900,1900,2000,2000,2000,2000,2000,...,800,800,800,800,700,700,700,700,800,700
